<a href="https://colab.research.google.com/github/LuchoValr/The-effect/blob/main/Cap16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Efectos fijos con el metodo "Absorbing the fixed effect"

In [1]:
!pip install causaldata
import numpy as np
import statsmodels.formula.api as sm
from causaldata import gapminder


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 27.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for causaldata: filename=causaldata-0.1.3-py3-none-any.whl size=2645603 sha256=1188678a4c6579e1535b70e238045f43e091dfdb0695e3e31ff054040af2836c
  Stored in directory: /root/.cache/pip/wheels/4a/8f/9f/13d3a7ab29b8a8c68f3e5dcc0fe00d0e8eef62fc508a20c7c6
Successfully built causaldata


In [8]:
gm = gapminder.load_pandas (). data
gm.head()

,country,continent,year,lifeExp,pop,gdpPercap
0,Afghanistan,Asia,1952,28.801,8425333,779.445314
1,Afghanistan,Asia,1957,30.332,9240934,820.853030
2,Afghanistan,Asia,1962,31.997,10267083,853.100710
3,Afghanistan,Asia,1967,34.020,11537966,836.197138
4,Afghanistan,Asia,1972,36.088,13079460,739.981106


In [9]:
# Put GDP per capita in log format since it's very skewed
gm['logGDPpercap'] = gm['gdpPercap'].apply('log')

In [5]:
# Use groupby to perform calculations by group
# Then use transform to subtract each variable's
# within -group mean to get within variation
gm[['logGDPpercap_within','lifeExp_within']] = (gm.groupby('country')[['logGDPpercap','lifeExp']].transform(lambda x: x - np.mean(x)))

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3472: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3472: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3472: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/usr/local/lib/python3.10/dist-packages/

In [6]:
# Analyze the within variation
m1 = sm.ols(formula = 'lifeExp_within ~ logGDPpercap_within ',data = gm).fit()

In [7]:
m1.summary ()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         lifeExp_within   R-squared:                       0.410
Model:                            OLS   Adj. R-squared:                  0.410
Method:                 Least Squares   F-statistic:                     1183.
Date:                Thu, 08 Jun 2023   Prob (F-statistic):          2.51e-197
Time:                        19:17:20   Log-Likelihood:                -5180.8
No. Observations:                1704   AIC:                         1.037e+04
Df Residuals:                    1702   BIC:                         1.038e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept            7.216e-16      0.123   5.88e-15      1.000      -0.241       0.241
logGDPpercap_within     9.7690      0.284     34.400      0.000       9.212      10.326
==============================================================================
Omnibus:                       23.105   Durbin-Watson:                   0.626
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               33.240
Skew:                          -0.143   Prob(JB):                     6.05e-08
Kurtosis:                       3.622   Cond. No.                         2.32
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

#Metodo con una variable binaria

In [10]:
gm = gapminder.load_pandas (). data

In [12]:
gm['logGDPpercap'] = gm['gdpPercap'].apply('log')

In [14]:
# Use C() to include binary variables for a categorical variable
m2 = sm.ols(formula = '''lifeExp ~ logGDPpercap + C(country)''', data = gm).fit()

In [15]:
m2.summary ()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                lifeExp   R-squared:                       0.846
Model:                            OLS   Adj. R-squared:                  0.832
Method:                 Least Squares   F-statistic:                     60.59
Date:                Thu, 08 Jun 2023   Prob (F-statistic):               0.00
Time:                        19:20:40   Log-Likelihood:                -5180.8
No. Observations:                1704   AIC:                         1.065e+04
Df Residuals:                    1561   BIC:                         1.143e+04
Df Model:                         142                                         
Covariance Type:            nonrobust                                         
==========================================================================================================
                                             coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------
Intercept                                -27.7735      2.501    -11.107      0.000     -32.678     -22.869
C(country)[T.Albania]                     17.7826      2.195      8.101      0.000      13.477      22.088
C(country)[T.Algeria]                      5.2411      2.214      2.367      0.018       0.897       9.585
C(country)[T.Angola]                     -13.9071      2.202     -6.316      0.000     -18.226      -9.588
C(country)[T.Argentina]                    8.1322      2.273      3.578      0.000       3.674      12.590
C(country)[T.Australia]                    6.4003      2.352      2.721      0.007       1.787      11.014
C(country)[T.Austria]                      5.1559      2.348      2.196      0.028       0.550       9.762
C(country)[T.Bahrain]                     -1.9682      2.344     -0.840      0.401      -6.566       2.629
C(country)[T.Bangladesh]                  12.4011      2.158      5.745      0.000       8.167      16.635
C(country)[T.Belgium]                      5.6169      2.349      2.391      0.017       1.009      10.225
C(country)[T.Benin]                        7.7450      2.161      3.584      0.000       3.506      11.984
C(country)[T.Bolivia]                      2.3380      2.193      1.066      0.286      -1.963       6.639
C(country)[T.Bosnia and Herzegovina]      17.3528      2.194      7.911      0.000      13.050      21.655
C(country)[T.Botswana]                     3.1522      2.200      1.433      0.152      -1.162       7.467
C(country)[T.Brazil]                       6.3181      2.230      2.833      0.005       1.944      10.692
C(country)[T.Bulgaria]                    12.6702      2.239      5.659      0.000       8.279      17.062
C(country)[T.Burkina Faso]                 6.8580      2.158      3.177      0.002       2.624      11.092
C(country)[T.Burundi]                     12.6464      2.164      5.843      0.000       8.401      16.892
C(country)[T.Cambodia]                    12.9167      2.160      5.981      0.000       8.680      17.153
C(country)[T.Cameroon]                     3.0620      2.171      1.411      0.159      -1.196       7.320
C(country)[T.Canada]                       5.4647      2.366      2.309      0.021       0.823      10.106
C(country)[T.Central African Republic]     4.7576      2.159      2.204      0.028       0.523       8.992
C(country)[T.Chad]                         5.7370      2.161      2.655      0.008       1.498       9.976
C(country)[T.Chile]                        9.9150      2.242      4.421      0.000       5.516      14.314
C(country)[T.China]                       21.2712      2.160      9.846      0.000      17.034      25.509
C(country)[T.Colombia]                    10.9197      2.209      4.943      0.000     

#Two-way fixed effects

In [17]:
!pip install linearmodels
import linearmodels as lm


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.3/82.3 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 4.8 MB/s eta 0:00:00


In [18]:
gm = gapminder.load_pandas (). data

In [20]:
gm['logGDPpercap'] = gm['gdpPercap'].apply('log')

In [22]:
# Set our individual and time (index) for our data
gm = gm.set_index (['country','year'])

In [23]:
# Specify the regression model
# And estimate with both sets of fixed effects
# EntityEffects and TimeEffects
# (this function can't handle more than two)
mod = lm.PanelOLS.from_formula(
    '''lifeExp ~ logGDPpercap + EntityEffects + TimeEffects''',gm)

In [24]:
twfe = mod.fit()

In [25]:
print(twfe)

                          PanelOLS Estimation Summary                           
Dep. Variable:                lifeExp   R-squared:                        0.0186
Estimator:                   PanelOLS   R-squared (Between):              0.3542
No. Observations:                1704   R-squared (Within):               0.1127
Date:                Thu, Jun 08 2023   R-squared (Overall):              0.3514
Time:                        19:33:03   Log-likelihood                   -4435.1
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      29.366
Entities:                         142   P-value                           0.0000
Avg Obs:                       12.000   Distribution:                  F(1,1550)
Min Obs:                       12.000                                           
Max Obs:                       12.000   F-statistic (robust):             29.366
                            

#Clustered standard errors

In [26]:
gm = gapminder.load_pandas (). data

In [27]:
gm['logGDPpercap'] = gm['gdpPercap'].apply('log')

In [28]:
# Set our individual and time (index) for our data
gm = gm.set_index (['country','year'])

In [29]:
mod = lm.PanelOLS.from_formula('''lifeExp ~ logGDPpercap + EntityEffects''',gm)

In [30]:
# Specify clustering when we fit the model
clfe = mod.fit(cov_type = 'clustered',
               cluster_entity = True)

In [31]:
print(clfe)

                          PanelOLS Estimation Summary                           
Dep. Variable:                lifeExp   R-squared:                        0.4101
Estimator:                   PanelOLS   R-squared (Between):              0.8786
No. Observations:                1704   R-squared (Within):               0.4101
Date:                Thu, Jun 08 2023   R-squared (Overall):              0.8731
Time:                        19:45:18   Log-likelihood                   -5180.8
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      1085.3
Entities:                         142   P-value                           0.0000
Avg Obs:                       12.000   Distribution:                  F(1,1561)
Min Obs:                       12.000                                           
Max Obs:                       12.000   F-statistic (robust):             195.30
                            